In [ ]:
import torch 
import pandas as pd 
import numpy as np
import os
from lincs_gsnn.data.DXDTDataset import DXDTDataset
from torch.utils.data import DataLoader 

from lincs_gsnn.models.SplineWeightEmbedding import SplineWeightEmbedding

from matplotlib import pyplot as plt
import seaborn as sbn
from sklearn.metrics import r2_score

import networkx as nx 
import pickle as pkl

from pypath.utils import mapping
import warnings 

from pypath.utils import mapping

warnings.filterwarnings("ignore")

from lincs_gsnn.models.ODEFunc import ODEFunc
from lincs_gsnn.data.TrajDataset import TrajDataset
from torchdiffeq import odeint 

import gseapy as gp
from lincs_gsnn.explain.viz import annotate_edges, get_drug_edges, make_subgraph, plot_graph, axes_to_gif

from lincs_gsnn.explain.infer_edge_weights import infer_edge_weights

import statsmodels.api as sm

from captum.attr import IntegratedGradients

%load_ext autoreload
%autoreload 2

In [3]:
data = torch.load('/home/teddy/local/lincs-gsnn/workflow/runs/default/bionetwork/bionetwork.pt', weights_only=False)
model = torch.load('../workflow/runs/default/pretrain/pretrained_model.pt', weights_only=False)
dxdt_scale = torch.load('../workflow/runs/default/pretrain/dxdt_scale.pt', weights_only=False).item()
x_names = pd.read_csv('/home/teddy/local/lincs-traj/workflow/runs/exp/default_v02/output/predict_grid/gene_names.csv')['gene_names'].values.astype(str)
dxdt_meta = pd.read_csv('/home/teddy/local/lincs-traj/workflow/runs/exp/default_v02/output/predict_grid/dxdt_meta.csv')

In [4]:
cond = dxdt_meta[lambda x: (x.cell_iname == 'BT20') & (x.pert_id == 'BRD-K49328571')]

target_genes = ['TP53']
target_ix = [x_names.tolist().index(x) for x in target_genes if x in x_names] 

obs_dir = '/home/teddy/local/lincs-traj/workflow/runs/exp/default_v02/output/predict_grid/dxdt/'


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

dataset = DXDTDataset(
                meta = cond, 
                obs_dir=obs_dir,
                scale=dxdt_scale,
                input_names=data.node_names_dict['input'],
                return_time=True)

loader = DataLoader(
    dataset,
    batch_size=64,
    shuffle=False,
    num_workers=1,
    persistent_workers=True)

x, dxdt, t = next(iter(loader)) 
x = x.to(device)
dxdt = dxdt.to(device)
t = t.to(device)   

In [11]:
x.shape

torch.Size([64, 997])

In [18]:
def func(edge_mask): 
    print(edge_mask.shape)
    dxdt = model(x, edge_mask=edge_mask)
    return dxdt 


In [19]:
model(x, edge_mask=None)

tensor([[ 0.0256,  0.0388,  0.2799,  ...,  0.0787, -0.1916, -0.2103],
        [ 0.0256,  0.0386,  0.2799,  ...,  0.0787, -0.1918, -0.2097],
        [ 0.0253,  0.0387,  0.2800,  ...,  0.0787, -0.1915, -0.2099],
        ...,
        [ 0.0254,  0.0388,  0.2798,  ...,  0.0787, -0.1914, -0.2102],
        [ 0.0256,  0.0385,  0.2799,  ...,  0.0788, -0.1917, -0.2101],
        [ 0.0254,  0.0386,  0.2799,  ...,  0.0788, -0.1915, -0.2099]],
       device='cuda:0', grad_fn=<IndexBackward0>)

In [20]:
integrated_gradients = IntegratedGradients(func)
edge_mask = torch.ones((x.shape[0], model.edge_index.shape[1],), device=device, requires_grad=True)
attributions_ig = integrated_gradients.attribute(edge_mask, target=0, n_steps=200, baselines=torch.zeros_like(edge_mask))

torch.Size([12800, 42564])


RuntimeError: The size of tensor a (64) must match the size of tensor b (12800) at non-singleton dimension 0